In [3]:
# 小米SU7微博数据 - 特征分析与预测模型
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from glob import glob
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 加载并合并所有数据
data_root = 'D:/010_CodePrograms/L/LLM_su7/小米SU7数据'

# 合并评论数据
comment_files = glob(f'{data_root}/*/热门微博评论数据.csv')
comments_list = []
for f in comment_files:
    df = pd.read_csv(f)
    df['crawl_date'] = os.path.basename(os.path.dirname(f))
    comments_list.append(df)
comments_df = pd.concat(comments_list, ignore_index=True)

# 合并评论链打分数据
chain_files = glob(f'{data_root}/*/评论链结构_打分.csv')
chains_list = []
for f in chain_files:
    df = pd.read_csv(f)
    df['crawl_date'] = os.path.basename(os.path.dirname(f))
    chains_list.append(df)
chains_df = pd.concat(chains_list, ignore_index=True)

print(f"评论数据: {len(comments_df):,} 条")
print(f"评论链数据: {len(chains_df):,} 条")
print(f"\n评论数据列: {list(comments_df.columns)}")
print(f"\n评论链数据列: {list(chains_df.columns)}")

评论数据: 273,975 条
评论链数据: 273,760 条

评论数据列: ['原文链接', '根评论ID', '父评论ID', '评论ID', '用户ID', '父用户昵称', '用户昵称', '评论内容', '发布时间', '子评论数', '点赞数', '用户认证', '用户总评论数', '用户总转发数', '用户总点赞数', '是否是一级评论', 'crawl_date']

评论链数据列: ['weibo_id', 'thread_id', 'time_id', 'comment_index', 'comment_user', 'replied_to_user', 'thread_text', '立场得分', '是否相关', 'crawl_date']


In [4]:
# 1. 重复数据分析
# ============================================
print("=" * 60)
print("【重复数据分析】")
print("=" * 60)

# 1.1 检查评论内容相同但点赞数/子评论数不同的情况
print("\n--- 1.1 相同评论内容的数据分布 ---")

# 按评论内容分组
content_groups = comments_df.groupby('评论内容').agg({
    '评论ID': 'count',
    '点赞数': ['min', 'max', 'mean', 'std'],
    '子评论数': ['min', 'max', 'mean', 'std'],
    'crawl_date': lambda x: list(x.unique())
}).reset_index()

content_groups.columns = ['评论内容', '出现次数', '点赞数_min', '点赞数_max', '点赞数_mean', '点赞数_std',
                          '子评论数_min', '子评论数_max', '子评论数_mean', '子评论数_std', '爬取日期']

# 筛选出现多次的评论
duplicates = content_groups[content_groups['出现次数'] > 1].copy()
print(f"总评论数: {len(comments_df):,}")
print(f"不重复评论内容数: {len(content_groups):,}")
print(f"出现多次的评论内容数: {len(duplicates):,}")

# 检查点赞数有变化的重复评论
duplicates['点赞数变化'] = duplicates['点赞数_max'] - duplicates['点赞数_min']
duplicates['子评论数变化'] = duplicates['子评论数_max'] - duplicates['子评论数_min']

likes_changed = duplicates[duplicates['点赞数变化'] > 0]
comments_changed = duplicates[duplicates['子评论数变化'] > 0]

print(f"\n点赞数有变化的重复评论: {len(likes_changed):,} ({len(likes_changed)/len(duplicates)*100:.1f}%)")
print(f"子评论数有变化的重复评论: {len(comments_changed):,} ({len(comments_changed)/len(duplicates)*100:.1f}%)")

【重复数据分析】

--- 1.1 相同评论内容的数据分布 ---


总评论数: 273,975
不重复评论内容数: 246,090
出现多次的评论内容数: 5,660

点赞数有变化的重复评论: 0 (0.0%)
子评论数有变化的重复评论: 1,063 (18.8%)


In [5]:
# 1.2 详细分析子评论数变化的情况
print("--- 1.2 子评论数变化详情 ---")

# 查看子评论数变化的分布
print("\n子评论数变化范围分布:")
print(comments_changed['子评论数变化'].describe())

# 查看一些具体例子
print("\n子评论数变化最大的5条评论:")
top_changes = comments_changed.nlargest(5, '子评论数变化')[['评论内容', '出现次数', '子评论数_min', '子评论数_max', '子评论数变化', '爬取日期']]
for idx, row in top_changes.iterrows():
    content = row['评论内容'][:50] + '...' if len(str(row['评论内容'])) > 50 else row['评论内容']
    print(f"\n内容: {content}")
    print(f"  出现次数: {row['出现次数']}, 子评论数: {row['子评论数_min']} → {row['子评论数_max']} (变化: +{row['子评论数变化']})")
    print(f"  爬取日期: {row['爬取日期']}")

# 1.3 分析重复原因
print("\n\n--- 1.3 重复数据的来源分析 ---")

# 检查是否是同一评论在不同日期被爬取
duplicates['跨日期'] = duplicates['爬取日期'].apply(lambda x: len(x) > 1)
cross_date = duplicates[duplicates['跨日期']]
same_date = duplicates[~duplicates['跨日期']]

print(f"跨日期重复的评论: {len(cross_date):,} ({len(cross_date)/len(duplicates)*100:.1f}%)")
print(f"同一日期内重复的评论: {len(same_date):,} ({len(same_date)/len(duplicates)*100:.1f}%)")

--- 1.2 子评论数变化详情 ---

子评论数变化范围分布:
count    1063.000000
mean       12.751646
std        75.358381
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max      1856.000000
Name: 子评论数变化, dtype: float64

子评论数变化最大的5条评论:

内容: 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的
  出现次数: 6, 子评论数: 0 → 1856 (变化: +1856)
  爬取日期: ['2025-04-01', '2025-04-02']

内容: 这个事故是buff叠满，特殊路况+车速快+智驾情况下松开双手+夜间行车+不是自己的车+分心疲劳驾驶+...
  出现次数: 4, 子评论数: 0 → 706 (变化: +706)
  爬取日期: ['2025-04-02', '2025-04-03']

内容: 97公里速度下的碰撞，撞完以后，我不觉得车内人员有能力开门，很可能当时就去世了，不去世也是重度昏迷
  出现次数: 3, 子评论数: 0 → 524 (变化: +524)
  爬取日期: ['2025-04-01']

内容: 交警处理啊，车企联系什么说什么
  出现次数: 2, 子评论数: 18 → 531 (变化: +513)
  爬取日期: ['2025-04-01']

内容: 就是可怜了随行的那两位女生了🙏🙏
  出现次数: 2, 子评论数: 0 → 493 (变化: +493)
  爬取日期: ['2025-04-02']


--- 1.3 重复数据的来源分析 ---
跨日期重复的评论: 3,830 (67.7%)
同一日期内重复的评论: 1,830 (32.3%)


In [6]:
# 1.4 重复数据处理建议
print("--- 1.4 重复数据处理建议 ---")

print("""
【分析结论】
1. 点赞数在重复记录中完全一致（0%变化）
   - 可能原因：爬虫在同一时间点多次爬取，或点赞数在短期内变化不大
   
2. 子评论数有18.8%存在变化
   - 说明同一条评论在不同时间被爬取，子评论会增加
   - 最大变化达1856条子评论
   
3. 重复来源：
   - 67.7%是跨日期重复（同一评论在多天被爬取）
   - 32.3%是同一天内重复（可能是爬虫问题或评论出现在多个热门微博下）

【处理建议】
- 不应简单删除重复，应保留子评论数最大（最新）的记录
- 或者保留所有记录，用于分析评论热度随时间的变化
""")

# 演示正确的去重方法
print("\n--- 推荐的去重方法 ---")
# 按评论ID去重，保留子评论数最大的记录
comments_dedup = comments_df.sort_values('子评论数', ascending=False).drop_duplicates(subset=['评论ID'], keep='first')
print(f"按评论ID去重后: {len(comments_dedup):,} 条 (原 {len(comments_df):,} 条)")

# 或者按评论内容去重
comments_dedup_content = comments_df.sort_values('子评论数', ascending=False).drop_duplicates(subset=['评论内容'], keep='first')
print(f"按评论内容去重后: {len(comments_dedup_content):,} 条")

--- 1.4 重复数据处理建议 ---

【分析结论】
1. 点赞数在重复记录中完全一致（0%变化）
   - 可能原因：爬虫在同一时间点多次爬取，或点赞数在短期内变化不大
   
2. 子评论数有18.8%存在变化
   - 说明同一条评论在不同时间被爬取，子评论会增加
   - 最大变化达1856条子评论
   
3. 重复来源：
   - 67.7%是跨日期重复（同一评论在多天被爬取）
   - 32.3%是同一天内重复（可能是爬虫问题或评论出现在多个热门微博下）

【处理建议】
- 不应简单删除重复，应保留子评论数最大（最新）的记录
- 或者保留所有记录，用于分析评论热度随时间的变化


--- 推荐的去重方法 ---
按评论ID去重后: 273,937 条 (原 273,975 条)


按评论内容去重后: 246,091 条


In [7]:
# 2. 话题标签分析
# ============================================
print("=" * 60)
print("【话题标签分析】")
print("=" * 60)

# 2.1 提取话题标签（#xxx#格式）
def extract_hashtags(text):
    if pd.isna(text):
        return []
    return re.findall(r'#([^#]+)#', str(text))

# 使用去重后的数据
comments_dedup['话题标签'] = comments_dedup['评论内容'].apply(extract_hashtags)

# 统计话题标签
all_hashtags = []
for tags in comments_dedup['话题标签']:
    all_hashtags.extend(tags)

hashtag_counts = Counter(all_hashtags)

print(f"\n--- 2.1 话题标签统计 ---")
print(f"包含话题标签的评论数: {sum(comments_dedup['话题标签'].apply(len) > 0):,}")
print(f"不同话题标签数: {len(hashtag_counts)}")
print(f"\nTOP 30 话题标签:")
for i, (tag, count) in enumerate(hashtag_counts.most_common(30), 1):
    print(f"{i:2}. #{tag}#: {count:,}")

【话题标签分析】



--- 2.1 话题标签统计 ---
包含话题标签的评论数: 890
不同话题标签数: 516

TOP 30 话题标签:
 1. #小米su7#: 48
 2. #小米SU7#: 31
 3. #小米#: 27
 4. #雷军发文纪念小米SU7上市一周年#: 23
 5. #小米SU7高速碰撞爆燃事件细节#: 22
 6. #胡侃#: 20
 7. #10元一辆的小米SU7卖爆了#: 19
 8. #小米su7[超话]#: 19
 9. #小米汽车[超话]#: 17
10. #3分钟复盘小米SU7事故#: 14
11. #小米su7高速碰撞爆燃事件细节#: 13
12. #小米SU7Ultra#: 13
13. #警方通报小米SU7湛江事故#: 12
14. #小米su7ultra#: 12
15. #小米汽车#: 12
16. #造车理念#: 12
17. #律师称10元纸扎版小米SU7或涉嫌侵权#: 12
18. #SU7爆燃遇难者父亲称小米仍没来联系#: 11
19. #雷军[超话]#: 11
20. #雷军回应小米SU7高速爆燃事故#: 11
21. #小米回应SU7高速公路车祸#: 11
22. #小米[超话]#: 11
23. #凤凰卫视诈骗#: 9
24. #小米客服称SU7电池双供应商随机装#: 9
25. #评测视频#: 9
26. #小米SU7事故专家质问2秒预警能干啥#: 9
27. #驾驶安全要紧握在自己手中#: 8
28. #小米辟谣信是假的#: 8
29. #这周的武汉属于雷军和小米汽车#: 8
30. #宁德时代称小米SU7用的不是其电池#: 8


In [8]:
# 2.2 话题标签与点赞数/子评论数的关系
print("--- 2.2 话题标签与互动指标的关系 ---")

# 创建有无话题标签的标记
comments_dedup['有话题标签'] = comments_dedup['话题标签'].apply(len) > 0

# 对比有无话题标签的评论
with_hashtag = comments_dedup[comments_dedup['有话题标签']]
without_hashtag = comments_dedup[~comments_dedup['有话题标签']]

print(f"\n有话题标签的评论: {len(with_hashtag):,} 条")
print(f"  点赞数 - 均值: {with_hashtag['点赞数'].mean():.2f}, 中位数: {with_hashtag['点赞数'].median():.0f}")
print(f"  子评论数 - 均值: {with_hashtag['子评论数'].mean():.2f}, 中位数: {with_hashtag['子评论数'].median():.0f}")

print(f"\n无话题标签的评论: {len(without_hashtag):,} 条")
print(f"  点赞数 - 均值: {without_hashtag['点赞数'].mean():.2f}, 中位数: {without_hashtag['点赞数'].median():.0f}")
print(f"  子评论数 - 均值: {without_hashtag['子评论数'].mean():.2f}, 中位数: {without_hashtag['子评论数'].median():.0f}")

# 统计检验
from scipy import stats
t_stat_likes, p_likes = stats.mannwhitneyu(with_hashtag['点赞数'], without_hashtag['点赞数'], alternative='two-sided')
t_stat_comments, p_comments = stats.mannwhitneyu(with_hashtag['子评论数'], without_hashtag['子评论数'], alternative='two-sided')

print(f"\n--- 统计检验 (Mann-Whitney U) ---")
print(f"点赞数差异 p值: {p_likes:.4e} {'(显著)' if p_likes < 0.05 else '(不显著)'}")
print(f"子评论数差异 p值: {p_comments:.4e} {'(显著)' if p_comments < 0.05 else '(不显著)'}")

--- 2.2 话题标签与互动指标的关系 ---

有话题标签的评论: 890 条
  点赞数 - 均值: 0.00, 中位数: 0
  子评论数 - 均值: 0.25, 中位数: 0

无话题标签的评论: 273,047 条
  点赞数 - 均值: 0.00, 中位数: 0
  子评论数 - 均值: 0.78, 中位数: 0

--- 统计检验 (Mann-Whitney U) ---
点赞数差异 p值: 1.0000e+00 (不显著)
子评论数差异 p值: 6.4065e-02 (不显著)


In [9]:
# 2.3 检查点赞数分布
print("--- 2.3 点赞数/子评论数分布检查 ---")

print("\n点赞数分布:")
print(comments_dedup['点赞数'].describe())
print(f"\n点赞数 > 0 的评论数: {(comments_dedup['点赞数'] > 0).sum():,}")
print(f"点赞数 = 0 的评论数: {(comments_dedup['点赞数'] == 0).sum():,}")

print("\n\n子评论数分布:")
print(comments_dedup['子评论数'].describe())
print(f"\n子评论数 > 0 的评论数: {(comments_dedup['子评论数'] > 0).sum():,}")

# 查看点赞数最高的评论
print("\n\n点赞数最高的10条评论:")
top_likes = comments_dedup.nlargest(10, '点赞数')[['评论内容', '点赞数', '子评论数']]
for idx, row in top_likes.iterrows():
    content = str(row['评论内容'])[:60] + '...' if len(str(row['评论内容'])) > 60 else row['评论内容']
    print(f"点赞:{row['点赞数']:5}, 子评论:{row['子评论数']:4} - {content}")

--- 2.3 点赞数/子评论数分布检查 ---

点赞数分布:
count    273937.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: 点赞数, dtype: float64

点赞数 > 0 的评论数: 0
点赞数 = 0 的评论数: 273,937


子评论数分布:
count    273937.000000
mean          0.776799
std          12.791707
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        2606.000000
Name: 子评论数, dtype: float64

子评论数 > 0 的评论数: 26,688


点赞数最高的10条评论:
点赞:    0, 子评论:2606 - 其实就是想借舆论诈一笔钱，希望小米别妥协，人道主义的钱也别给
点赞:    0, 子评论:1954 - 车提醒了好几次，什么注意力不集中，什么手离方向盘，明显是开车的人不好好开车造成的事故
点赞:    0, 子评论:1856 - 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的
点赞:    0, 子评论:1708 - 史上首次，驾驶员违法造成事故，追究车企[doge][喵喵][二哈]
点赞:    0, 子评论:1294 - 重点是 小米的智驾很差很差 导致这个事故
点赞:    0, 子评论:1108 - 那这么说 都跟车主联系上了 没教她们怎么开门逃生吗？？
点赞:    0, 子评论:1039 - 没义务，但小米自己公告说有联系，自相矛盾
点赞:    0, 子评论: 850 - 第五点，意思是“车门打不开“的结论她是猜的？
点赞:    0, 子评论: 794 - 电车出交通事故就找车企。自己怎么驾驶的呢？
点赞:    0, 子评论: 771 - 事情有结果了，或者证据不利


In [10]:
# 2.4 话题标签分析总结
print("--- 2.4 话题标签分析总结 ---")
print("""
【发现】
1. 点赞数字段全部为0，数据爬取存在问题，无法用于预测
2. 子评论数有效，可作为预测目标
3. 仅0.33%的评论(890/273937)包含话题标签
4. 话题标签主要与小米汽车/SU7相关的新闻事件有关

【话题标签类型】
- 产品相关: #小米SU7#, #小米汽车#, #小米SU7Ultra#
- 事件相关: #小米SU7高速碰撞爆燃事件细节#, #警方通报小米SU7湛江事故#
- 人物相关: #雷军#, #胡侃#
- 舆论相关: #雷军回应小米SU7高速爆燃事故#

【预测价值评估】
- 话题标签覆盖率太低(0.33%)，对预测的帮助有限
- 但有话题标签的评论可能具有不同特征
- 建议：作为辅助特征使用，但不应作为主要预测依据
""")

# 将小米汽车相关标签合并统计
xiaomi_tags = ['小米su7', '小米SU7', '小米', '小米汽车', '小米SU7Ultra', '小米su7ultra']
comments_dedup['小米相关标签'] = comments_dedup['话题标签'].apply(
    lambda tags: any(any(xt.lower() in t.lower() for xt in xiaomi_tags) for t in tags)
)
print(f"\n包含小米相关话题标签的评论数: {comments_dedup['小米相关标签'].sum():,}")

--- 2.4 话题标签分析总结 ---

【发现】
1. 点赞数字段全部为0，数据爬取存在问题，无法用于预测
2. 子评论数有效，可作为预测目标
3. 仅0.33%的评论(890/273937)包含话题标签
4. 话题标签主要与小米汽车/SU7相关的新闻事件有关

【话题标签类型】
- 产品相关: #小米SU7#, #小米汽车#, #小米SU7Ultra#
- 事件相关: #小米SU7高速碰撞爆燃事件细节#, #警方通报小米SU7湛江事故#
- 人物相关: #雷军#, #胡侃#
- 舆论相关: #雷军回应小米SU7高速爆燃事故#

【预测价值评估】
- 话题标签覆盖率太低(0.33%)，对预测的帮助有限
- 但有话题标签的评论可能具有不同特征
- 建议：作为辅助特征使用，但不应作为主要预测依据


包含小米相关话题标签的评论数: 569


In [11]:
# 3. 特征工程
# ============================================
print("=" * 60)
print("【特征工程】")
print("=" * 60)

# 使用去重后的评论数据
df = comments_dedup.copy()

# 3.1 评论长度
df['评论长度'] = df['评论内容'].fillna('').apply(len)

# 3.2 标点符号特征
def count_punctuation(text):
    if pd.isna(text):
        return 0, 0, 0
    text = str(text)
    exclamation = text.count('！') + text.count('!')
    question = text.count('？') + text.count('?')
    emoji_pattern = r'\[[\u4e00-\u9fa5a-zA-Z]+\]'  # 微博表情 [xxx]
    emoji_count = len(re.findall(emoji_pattern, text))
    return exclamation, question, emoji_count

punct_features = df['评论内容'].apply(count_punctuation)
df['感叹号数'] = punct_features.apply(lambda x: x[0])
df['问号数'] = punct_features.apply(lambda x: x[1])
df['表情数'] = punct_features.apply(lambda x: x[2])

# 3.3 时间特征
df['发布时间'] = pd.to_datetime(df['发布时间'], errors='coerce')
df['发布小时'] = df['发布时间'].dt.hour
df['发布星期'] = df['发布时间'].dt.dayofweek  # 0=周一, 6=周日
df['是否工作日'] = df['发布星期'].apply(lambda x: 1 if x < 5 else 0)

# 3.4 用户特征
df['用户活跃度'] = df['用户总评论数'] + df['用户总转发数'] + df['用户总点赞数']
df['是否认证用户'] = df['用户认证'].apply(lambda x: 1 if pd.notna(x) and x != '' else 0)

# 3.5 是否为一级评论
df['是一级评论'] = df['是否是一级评论'].apply(lambda x: 1 if x else 0)

print("特征工程完成!")
print(f"\n生成的特征: 评论长度, 感叹号数, 问号数, 表情数, 发布小时, 发布星期, 是否工作日, 用户活跃度, 是否认证用户, 是一级评论, 有话题标签")

【特征工程】


特征工程完成!

生成的特征: 评论长度, 感叹号数, 问号数, 表情数, 发布小时, 发布星期, 是否工作日, 用户活跃度, 是否认证用户, 是一级评论, 有话题标签


In [12]:
# 3.6 合并立场得分（情感特征）
print("--- 3.6 合并立场得分 ---")

# 评论链数据中的立场得分
chains_dedup = chains_df.drop_duplicates(subset=['thread_text'], keep='first')
print(f"评论链去重后: {len(chains_dedup):,} 条")

# 查看立场得分分布
print(f"\n立场得分分布:")
print(chains_dedup['立场得分'].describe())
print(f"\n是否相关分布:")
print(chains_dedup['是否相关'].value_counts())

# 计算情感强度（立场得分的绝对值）
chains_dedup = chains_dedup.copy()
chains_dedup['情感强度'] = chains_dedup['立场得分'].abs()

# 将立场得分和是否相关合并到评论数据
# 使用评论内容进行匹配
df_with_stance = df.merge(
    chains_dedup[['thread_text', '立场得分', '是否相关', '情感强度']],
    left_on='评论内容',
    right_on='thread_text',
    how='left'
)

print(f"\n合并后数据量: {len(df_with_stance):,}")
print(f"有立场得分的评论: {df_with_stance['立场得分'].notna().sum():,}")

--- 3.6 合并立场得分 ---
评论链去重后: 244,654 条

立场得分分布:
count    239253.000000
mean         -0.168711
std           0.435182
min          -1.000000
25%          -0.456000
50%          -0.156000
75%           0.000000
max           1.000000
Name: 立场得分, dtype: float64

是否相关分布:
是否相关
1.0    189914
0.0     49339
Name: count, dtype: int64



合并后数据量: 273,937
有立场得分的评论: 266,871


In [13]:
# 4. 相关性分析
# ============================================
print("=" * 60)
print("【特征与子评论数的相关性分析】")
print("=" * 60)

# 准备分析用的特征
feature_cols = ['评论长度', '感叹号数', '问号数', '表情数', '发布小时', 
                '是否工作日', '用户活跃度', '是否认证用户', '是一级评论', 
                '有话题标签', '立场得分', '情感强度', '是否相关']

# 过滤有效数据
df_analysis = df_with_stance[feature_cols + ['子评论数']].dropna()
print(f"用于分析的数据量: {len(df_analysis):,}")

# 计算与子评论数的相关系数
correlations = {}
for col in feature_cols:
    # Spearman相关（适用于非正态分布）
    corr, p_value = stats.spearmanr(df_analysis[col], df_analysis['子评论数'])
    correlations[col] = {'相关系数': corr, 'p值': p_value}

# 排序并显示
corr_df = pd.DataFrame(correlations).T
corr_df = corr_df.sort_values('相关系数', key=abs, ascending=False)

print("\n--- 与子评论数的Spearman相关系数 ---")
print("(按绝对值排序)")
for idx, row in corr_df.iterrows():
    sig = '***' if row['p值'] < 0.001 else ('**' if row['p值'] < 0.01 else ('*' if row['p值'] < 0.05 else ''))
    print(f"{idx:12}: {row['相关系数']:+.4f} {sig}")

【特征与子评论数的相关性分析】
用于分析的数据量: 266,871



--- 与子评论数的Spearman相关系数 ---
(按绝对值排序)
是一级评论       : +0.2811 ***
表情数         : +0.1090 ***
评论长度        : +0.0939 ***
用户活跃度       : +0.0303 ***
发布小时        : +0.0174 ***
问号数         : +0.0152 ***
立场得分        : +0.0128 ***
感叹号数        : +0.0059 **
有话题标签       : -0.0036 
是否相关        : -0.0035 
情感强度        : +0.0016 
是否工作日       : +0.0010 
是否认证用户      : +nan 


In [14]:
# 4.2 相关性可视化
print("--- 4.2 相关性热力图 ---")

# 选择主要特征
main_features = ['评论长度', '感叹号数', '问号数', '表情数', '发布小时', 
                 '是一级评论', '立场得分', '情感强度', '子评论数']

corr_matrix = df_analysis[main_features].corr(method='spearman')

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, ax=ax, vmin=-0.3, vmax=0.3)
plt.tight_layout()
plt.savefig('D:/010_CodePrograms/L/LLM_su7/results/feature_correlation.png', dpi=200, bbox_inches='tight')
plt.close()
print("相关性热力图已保存至 results/feature_correlation.png")

# 4.3 关键特征与子评论数的关系
print("\n--- 4.3 关键发现 ---")
print("""
【相关性分析结论】
1. 是一级评论 (r=0.281): 最强正相关
   - 一级评论更容易获得回复

2. 表情数 (r=0.109): 较强正相关
   - 使用更多表情的评论更容易引发讨论

3. 评论长度 (r=0.094): 中等正相关
   - 较长的评论内容更丰富，更易引发讨论

4. 立场得分/情感强度: 弱相关
   - 情感倾向对子评论数影响不大

5. 话题标签: 几乎无相关
   - 话题标签对引发讨论没有显著影响
""")

--- 4.2 相关性热力图 ---


相关性热力图已保存至 results/feature_correlation.png

--- 4.3 关键发现 ---

【相关性分析结论】
1. 是一级评论 (r=0.281): 最强正相关
   - 一级评论更容易获得回复

2. 表情数 (r=0.109): 较强正相关
   - 使用更多表情的评论更容易引发讨论

3. 评论长度 (r=0.094): 中等正相关
   - 较长的评论内容更丰富，更易引发讨论

4. 立场得分/情感强度: 弱相关
   - 情感倾向对子评论数影响不大

5. 话题标签: 几乎无相关
   - 话题标签对引发讨论没有显著影响



In [15]:
# 5. 构建预测模型
# ============================================
print("=" * 60)
print("【构建子评论数预测模型】")
print("=" * 60)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import gc

# 5.1 准备数据
feature_cols_model = ['评论长度', '感叹号数', '问号数', '表情数', '发布小时', 
                      '是否工作日', '是一级评论', '立场得分', '情感强度', '是否相关']

# 过滤有效数据并采样（减少内存占用）
df_model = df_with_stance[feature_cols_model + ['子评论数']].dropna()
print(f"有效数据量: {len(df_model):,}")

# 对子评论数进行log变换（处理长尾分布）
df_model['子评论数_log'] = np.log1p(df_model['子评论数'])

# 划分训练集和测试集
X = df_model[feature_cols_model]
y = df_model['子评论数_log']  # 使用log变换后的目标

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"训练集: {len(X_train):,}, 测试集: {len(X_test):,}")

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

【构建子评论数预测模型】


有效数据量: 266,871
训练集: 213,496, 测试集: 53,375


In [16]:
# 5.2 训练多个模型
print("--- 5.2 模型训练与评估 ---")

results = {}

# 模型1: Ridge回归
print("\n训练 Ridge 回归...")
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)
results['Ridge'] = {
    'MAE': mean_absolute_error(y_test, y_pred_ridge),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_ridge)),
    'R2': r2_score(y_test, y_pred_ridge)
}

# 模型2: 随机森林（使用较小的参数）
print("训练 随机森林...")
rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)  # 随机森林不需要标准化
y_pred_rf = rf.predict(X_test)
results['RandomForest'] = {
    'MAE': mean_absolute_error(y_test, y_pred_rf),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'R2': r2_score(y_test, y_pred_rf)
}

# 模型3: 梯度提升（轻量级）
print("训练 梯度提升...")
gb = GradientBoostingRegressor(n_estimators=50, max_depth=5, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
results['GradientBoosting'] = {
    'MAE': mean_absolute_error(y_test, y_pred_gb),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_gb)),
    'R2': r2_score(y_test, y_pred_gb)
}

# 显示结果
print("\n" + "=" * 50)
print("模型性能对比 (预测log(子评论数+1))")
print("=" * 50)
results_df = pd.DataFrame(results).T
print(results_df.round(4))

--- 5.2 模型训练与评估 ---

训练 Ridge 回归...
训练 随机森林...


训练 梯度提升...



模型性能对比 (预测log(子评论数+1))
                     MAE    RMSE      R2
Ridge             0.2330  0.4760  0.0634
RandomForest      0.2095  0.4708  0.0837
GradientBoosting  0.2103  0.4704  0.0856


In [17]:
# 5.3 特征重要性分析
print("--- 5.3 特征重要性 ---")

# 随机森林特征重要性
importance_rf = pd.DataFrame({
    '特征': feature_cols_model,
    '重要性': rf.feature_importances_
}).sort_values('重要性', ascending=False)

print("\n随机森林特征重要性:")
for idx, row in importance_rf.iterrows():
    bar = '█' * int(row['重要性'] * 50)
    print(f"{row['特征']:12}: {row['重要性']:.4f} {bar}")

# 梯度提升特征重要性
importance_gb = pd.DataFrame({
    '特征': feature_cols_model,
    '重要性': gb.feature_importances_
}).sort_values('重要性', ascending=False)

print("\n梯度提升特征重要性:")
for idx, row in importance_gb.iterrows():
    bar = '█' * int(row['重要性'] * 50)
    print(f"{row['特征']:12}: {row['重要性']:.4f} {bar}")

--- 5.3 特征重要性 ---

随机森林特征重要性:
是一级评论       : 0.4306 █████████████████████
评论长度        : 0.2773 █████████████
立场得分        : 0.0744 ███
发布小时        : 0.0672 ███
情感强度        : 0.0595 ██
问号数         : 0.0356 █
表情数         : 0.0281 █
是否工作日       : 0.0112 
感叹号数        : 0.0103 
是否相关        : 0.0058 

梯度提升特征重要性:
是一级评论       : 0.5644 ████████████████████████████
评论长度        : 0.3045 ███████████████
问号数         : 0.0290 █
发布小时        : 0.0279 █
表情数         : 0.0230 █
立场得分        : 0.0166 
情感强度        : 0.0160 
是否工作日       : 0.0106 
感叹号数        : 0.0059 
是否相关        : 0.0021 


In [18]:
# 5.4 预测效果可视化
print("--- 5.4 预测效果可视化 ---")

# 将预测值转换回原始尺度
y_test_original = np.expm1(y_test)
y_pred_gb_original = np.expm1(y_pred_gb)

# 绘制实际值vs预测值
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：散点图
ax1 = axes[0]
ax1.scatter(y_test_original, y_pred_gb_original, alpha=0.1, s=1)
ax1.plot([0, 100], [0, 100], 'r--', label='完美预测')
ax1.set_xlabel('实际子评论数')
ax1.set_ylabel('预测子评论数')
ax1.set_xlim(0, 100)
ax1.set_ylim(0, 20)
ax1.legend()

# 右图：残差分布
ax2 = axes[1]
residuals = y_test_original - y_pred_gb_original
ax2.hist(residuals, bins=100, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='r', linestyle='--')
ax2.set_xlabel('残差 (实际 - 预测)')
ax2.set_ylabel('频数')
ax2.set_xlim(-50, 50)

plt.tight_layout()
plt.savefig('D:/010_CodePrograms/L/LLM_su7/results/prediction_evaluation.png', dpi=200, bbox_inches='tight')
plt.close()
print("预测效果图已保存至 results/prediction_evaluation.png")

--- 5.4 预测效果可视化 ---


预测效果图已保存至 results/prediction_evaluation.png


In [19]:
# 6. 总结
# ============================================
print("=" * 60)
print("【分析总结】")
print("=" * 60)
print("""
=== 1. 重复数据分析 ===
• 重复评论内容: 5,660条 (占比2.3%)
• 点赞数在重复记录中完全一致 (数据可能有问题)
• 子评论数在18.8%的重复记录中有变化
• 67.7%的重复是跨日期的同一评论
• 建议: 按评论ID去重，保留子评论数最大的记录

=== 2. 话题标签分析 ===
• 仅0.33%的评论包含话题标签 (890条)
• 主要是小米汽车相关的新闻事件标签
• 话题标签与子评论数几乎无相关 (r=-0.004)
• 结论: 话题标签对预测几乎没有帮助

=== 3. 特征相关性 ===
按重要性排序:
1. 是一级评论 (r=0.281) - 最重要特征
2. 表情数 (r=0.109) - 次要特征
3. 评论长度 (r=0.094) - 中等重要
4. 立场得分/情感强度 - 弱相关
5. 时间特征 - 较弱影响

=== 4. 预测模型 ===
• 最佳模型: 梯度提升 (R²=0.086)
• 特征重要性: 是一级评论(56%) > 评论长度(30%)
• 模型解释力有限，说明子评论数受很多隐性因素影响
  (如: 评论内容质量、发布时机、原微博热度等)

=== 5. 数据问题 ===
• 点赞数字段全部为0，无法使用
• 只能预测子评论数
""")

# 保存分析结果
summary_data = {
    '指标': ['总评论数', '去重后评论数', '有话题标签评论', '有立场得分评论', 
             '最佳模型R²', '最重要特征'],
    '值': [f'{len(comments_df):,}', f'{len(comments_dedup):,}', 
           f'{890} (0.33%)', f'{266871} (97.4%)',
           '0.086 (梯度提升)', '是一级评论 (56%)']
}
pd.DataFrame(summary_data).to_csv('D:/010_CodePrograms/L/LLM_su7/results/feature_analysis_summary.csv', 
                                   index=False, encoding='utf-8-sig')
print("\n分析摘要已保存至 results/feature_analysis_summary.csv")

【分析总结】

=== 1. 重复数据分析 ===
• 重复评论内容: 5,660条 (占比2.3%)
• 点赞数在重复记录中完全一致 (数据可能有问题)
• 子评论数在18.8%的重复记录中有变化
• 67.7%的重复是跨日期的同一评论
• 建议: 按评论ID去重，保留子评论数最大的记录

=== 2. 话题标签分析 ===
• 仅0.33%的评论包含话题标签 (890条)
• 主要是小米汽车相关的新闻事件标签
• 话题标签与子评论数几乎无相关 (r=-0.004)
• 结论: 话题标签对预测几乎没有帮助

=== 3. 特征相关性 ===
按重要性排序:
1. 是一级评论 (r=0.281) - 最重要特征
2. 表情数 (r=0.109) - 次要特征
3. 评论长度 (r=0.094) - 中等重要
4. 立场得分/情感强度 - 弱相关
5. 时间特征 - 较弱影响

=== 4. 预测模型 ===
• 最佳模型: 梯度提升 (R²=0.086)
• 特征重要性: 是一级评论(56%) > 评论长度(30%)
• 模型解释力有限，说明子评论数受很多隐性因素影响
  (如: 评论内容质量、发布时机、原微博热度等)

=== 5. 数据问题 ===
• 点赞数字段全部为0，无法使用
• 只能预测子评论数


分析摘要已保存至 results/feature_analysis_summary.csv


In [20]:
# 查看链接字段
print("=" * 60)
print("【查看评论链接字段】")
print("=" * 60)

print("\n评论数据列名:")
print(comments_df.columns.tolist())

print("\n原文链接样例 (前10条):")
for i, link in enumerate(comments_df['原文链接'].head(10)):
    print(f"{i+1}. {link}")

print(f"\n原文链接非空数量: {comments_df['原文链接'].notna().sum():,}")
print(f"原文链接唯一值数量: {comments_df['原文链接'].nunique():,}")

【查看评论链接字段】

评论数据列名:
['原文链接', '根评论ID', '父评论ID', '评论ID', '用户ID', '父用户昵称', '用户昵称', '评论内容', '发布时间', '子评论数', '点赞数', '用户认证', '用户总评论数', '用户总转发数', '用户总点赞数', '是否是一级评论', 'crawl_date']

原文链接样例 (前10条):
1. https://weibo.com/7871239944/PkvsRdOvf
2. https://weibo.com/7871239944/PkvsRdOvf
3. https://weibo.com/7871239944/PkvsRdOvf
4. https://weibo.com/7871239944/PkvsRdOvf
5. https://weibo.com/7871239944/PkvsRdOvf
6. https://weibo.com/7871239944/PkvsRdOvf
7. https://weibo.com/7871239944/PkvsRdOvf
8. https://weibo.com/7871239944/PkvsRdOvf
9. https://weibo.com/7871239944/PkvsRdOvf
10. https://weibo.com/7871239944/PkvsRdOvf

原文链接非空数量: 273,975
原文链接唯一值数量: 4,079


In [21]:
# 查看热门微博数据中的指标
print("=" * 60)
print("【查看热门微博数据】")
print("=" * 60)

# 加载热门微博数据
weibo_files = glob(f'{data_root}/*/热门微博.csv')
weibo_list = []
for f in weibo_files:
    df = pd.read_csv(f)
    df['crawl_date'] = os.path.basename(os.path.dirname(f))
    weibo_list.append(df)
weibo_df = pd.concat(weibo_list, ignore_index=True)

print(f"热门微博数据: {len(weibo_df):,} 条")
print(f"\n热门微博列名:")
print(weibo_df.columns.tolist())

print("\n热门微博数据样例:")
print(weibo_df[['微博链接', '转发数', '评论数', '点赞数']].head(10))

【查看热门微博数据】


热门微博数据: 7,215 条

热门微博列名:
['author_name', 'author_url', 'weibo_url', 'publish_time', 'weibo_content', 'repost_count', 'comment_count', 'like_count', 'crawl_date']

热门微博数据样例:


KeyError: "None of [Index(['微博链接', '转发数', '评论数', '点赞数'], dtype='object')] are in the [columns]"

In [22]:
# 查看热门微博数据的实际字段
print("热门微博数据样例:")
print(weibo_df[['weibo_url', 'repost_count', 'comment_count', 'like_count']].head(10))

print("\n\n转发数/评论数/点赞数分布:")
print(weibo_df[['repost_count', 'comment_count', 'like_count']].describe())

# 检查是否有有效数据
print(f"\n转发数>0: {(weibo_df['repost_count'] > 0).sum():,}")
print(f"评论数>0: {(weibo_df['comment_count'] > 0).sum():,}")
print(f"点赞数>0: {(weibo_df['like_count'] > 0).sum():,}")

热门微博数据样例:
                                weibo_url  repost_count  comment_count  \
0  https://weibo.com/7871239944/PkvsRdOvf            48            105   
1  https://weibo.com/7747056179/Pkpf5e7ul            30            280   
2  https://weibo.com/1653076903/PkrMwbgDE            42            463   
3  https://weibo.com/3873096802/PktNnatj8            24             21   
4  https://weibo.com/7322674601/Pks4m9ymM            16            684   
5  https://weibo.com/1979911501/PkrUelYYx            17            529   
6  https://weibo.com/1080548507/PkwaG1lEN             3             30   
7  https://weibo.com/5833359023/PkrytDSdo             8             88   
8  https://weibo.com/1711672842/Pkx24DkpV             4             17   
9  https://weibo.com/7350628330/PktDZDFfl             0              7   

   like_count  
0           0  
1           0  
2           0  
3           0  
4           0  
5           0  
6           0  
7           0  
8           0  
9           0  

In [23]:
# 确认需要爬取的数据
print("=" * 60)
print("【数据问题确认】")
print("=" * 60)

print("""
【现有数据情况】
1. 热门微博数据:
   - 转发数: 有数据 ✓
   - 评论数: 有数据 ✓  
   - 点赞数: 全部为0 ✗

2. 评论数据:
   - 点赞数: 全部为0 ✗
   - 子评论数: 有数据 ✓

【需要重新爬取】
- 热门微博的点赞数
- 每条评论的点赞数
""")

# 获取需要爬取的唯一微博链接
unique_weibo_urls = comments_df['原文链接'].unique()
print(f"需要爬取的唯一微博链接数: {len(unique_weibo_urls):,}")

# 获取需要爬取的唯一评论ID
unique_comment_ids = comments_df['评论ID'].unique()
print(f"需要爬取的唯一评论ID数: {len(unique_comment_ids):,}")

【数据问题确认】

【现有数据情况】
1. 热门微博数据:
   - 转发数: 有数据 ✓
   - 评论数: 有数据 ✓  
   - 点赞数: 全部为0 ✗

2. 评论数据:
   - 点赞数: 全部为0 ✗
   - 子评论数: 有数据 ✓

【需要重新爬取】
- 热门微博的点赞数
- 每条评论的点赞数

需要爬取的唯一微博链接数: 4,079
需要爬取的唯一评论ID数: 273,937


In [5]:
# 分析失败链接的特征
print("=" * 60)
print("【失败链接分析 - 无需Cookie】")
print("=" * 60)

# 从爬取日志分析：no_data 表示API返回成功但没有微博数据
# 这通常意味着微博已被删除或设为私密

print("""
【API返回分析】
当前没有Cookie无法直接验证，但根据爬取时的结果：

1. status='success' (2,779条, 68.1%)
   - API返回了 reposts_count, comments_count, attitudes_count
   - 微博正常可访问

2. status='no_data' (1,299条, 31.8%)
   - API返回200但数据中没有微博信息
   - 可能原因：
     a) 微博已被作者删除
     b) 微博被设为仅自己可见/好友可见
     c) 账号被封禁
     d) 微博因违规被隐藏

3. status='error' (1条, 0.0%)
   - 网络超时导致请求失败
""")

# 分析失败链接的用户ID分布
failed_user_ids = []
for url in no_data_links:
    match = re.search(r'weibo\.com/(\d+)/', url)
    if match:
        failed_user_ids.append(match.group(1))

from collections import Counter
user_counter = Counter(failed_user_ids)

print("\n【失败链接用户分布 TOP 20】")
print("(同一用户多条微博失败，可能是账号问题)")
print("-" * 40)
for user_id, count in user_counter.most_common(20):
    print(f"用户ID {user_id}: {count} 条失败")

print(f"\n涉及 {len(user_counter)} 个不同用户")

【失败链接分析 - 无需Cookie】

【API返回分析】
当前没有Cookie无法直接验证，但根据爬取时的结果：

1. status='success' (2,779条, 68.1%)
   - API返回了 reposts_count, comments_count, attitudes_count
   - 微博正常可访问

2. status='no_data' (1,299条, 31.8%)
   - API返回200但数据中没有微博信息
   - 可能原因：
     a) 微博已被作者删除
     b) 微博被设为仅自己可见/好友可见
     c) 账号被封禁
     d) 微博因违规被隐藏

3. status='error' (1条, 0.0%)
   - 网络超时导致请求失败


【失败链接用户分布 TOP 20】
(同一用户多条微博失败，可能是账号问题)
----------------------------------------
用户ID 2017327511: 57 条失败
用户ID 6027322470: 41 条失败
用户ID 1841017701: 38 条失败
用户ID 7348359537: 24 条失败
用户ID 1720464350: 23 条失败
用户ID 6268515072: 20 条失败
用户ID 5572566998: 19 条失败
用户ID 7603251011: 17 条失败
用户ID 2827386331: 15 条失败
用户ID 5556519481: 13 条失败
用户ID 6815032120: 12 条失败
用户ID 1677920540: 12 条失败
用户ID 2743487997: 12 条失败
用户ID 5801748561: 11 条失败
用户ID 3163693955: 11 条失败
用户ID 5714719345: 11 条失败
用户ID 5147769836: 11 条失败
用户ID 2323195050: 10 条失败
用户ID 5386924056: 10 条失败
用户ID 6619788362: 10 条失败

涉及 507 个不同用户
